In [1]:
import nltk

train = open('datatrain.txt','w')
sentences = open('sentences.txt').read().lower()
token = nltk.word_tokenize(sentences)
totalWord={}


def dataread(): #create the POSTagging list
    dict = {}
    tahap = 0
    dict['s.t.r'] ={}
    dict['s.t.r']['start'] = tahap
    with open('input_text.txt') as data:
        while tahap<=500:
            sent = data.readline()
            sent = sent.lower()
            if len(sent.strip()) == 0:
                continue
            if '#eot#' in sent or tahap == 101:
                break
            sentence = sent
            if ('#' in sent):
                tahap +=1
                continue
            sentence = sentence.split("\t")
            if (sentence[2]=="_"):
                tag = sentence[3]
            else:
                tag = sentence[2]
            if (sentence[1] in dict):
                if (tag in dict[sentence[1]]):
                    dict[sentence[1]][tag] +=1
                else:
                    dict[sentence[1]][tag] = 1
            else:
                dict[sentence[1]] ={}
                dict[sentence[1]][tag] = 1
        dict['s.t.r']['start'] = tahap
    return dict

In [3]:
def datawrite(train): #USE WHOLE SENTENCE as Data Train
    tahap = 0
    loop = True
    with open('input_text.txt') as data:
        while tahap <=500:
            sent = data.readline()
            sent = sent.lower()
            if len(sent.strip()) == 0:
                continue
            if tahap == 4477:
                break
                loop == False
            sentence = sent
            if ('# sent_id' in sent):
                sent = data.readline()
                sentence = sent.split("text = ")[1]
                train.write('s.t.r '+sentence.lower())
                tahap +=1


datawrite(train)
newfile = open('datatrain.txt').read()
newfile = newfile.replace("'"," ' ").replace('2012.','2012 .')
tokens = nltk.word_tokenize(newfile)

In [4]:
def count(word): #Count the amount of each occurrence words
    countword ={}
    num = 0
    for i in tokens:
        if (i == '``') or (i == "''"):
            i = '"'
        if i == word:
            if word in countword:
                countword[word] += 1
                num +=1
            else:
                countword[word] = 1
                num =+1
        else:
            pass
    return num

In [5]:
def emiProb(dict):
    prob = {}
    for k,v in dict.items():
        for k2, v2 in dict[k].items():
            if k not in prob:
                prob[k] ={}
                if k  == 's.t.r':
                    prob[k] = {}
                    prob[k]['start'] =1
                if (count(k) == 0):
                    #BONUS, smoothing is did in here
                    prob[k][k2] = 1
                else:
                    #print('kata',k,'tag',k2,dict[k][k2])
                    #print('huruf',k,'ada',count(k))
                    prob[k][k2] = dict[k][k2]/count(k)
            else:
                if k2 not in prob[k]:
                    prob[k][k2] = dict[k][k2]/count(k)
    return prob

def countTag(dict,tag): #Create a POSTagger based on highest emission probability table.
    tagCount = {}
    num = 0
    for word in tokens:
        if (word == '``') or (word == "''"):
            word = '"'
        if word not in dict:
            continue
        for a,b in dict[word].items():
            if a == tag:
                if a not in tagCount:
                    tagCount[a] =1
                    num +=1
                else:
                    tagCount[a] +=1
                    num +=1
            else:
                pass
    return num


def postagg(dict,emiTag): #Create a POSTagger based on highest emission probability table.
    for i, j in emiTag.items():
        if len(j) > 1:
            print('The Ambiguity words',i,'found words =',len(j))
    hiEmi = {}
    for k,v in dict.items():
        for k2, v2 in dict[k].items():
            if k not in hiEmi:
                hiEmi[k] ={}
                hiEmi[k][k2] = emiTag[k][k2]
            else:
                if len(emiTag[k]) >1:
                    hiEmi[k]={}
                    t = max(emiTag[k])
                    hiEmi[k][t] = emiTag[k][t]
    return hiEmi

def transition(dict): #Create a tag transition probability table (bigram model, only record one previous tag)
    bigramCount = {}
    temp = {}
    length = len(tokens)
    for i in range(length-1):
        word1 = tokens[i]
        word2 = tokens[i+1]
        if word1 == '``' or word1 == "''":
            word1 = '"'
        if word2 == '``' or word2 == "''":
            word2 = '"'
        if word1 == 'm.':
            word1 = 'm'
        if word2 == 'm.':
            word2 = 'm'
        if word1 == 'pepper':
            word1 = "pepper's"
        if word2 == 'pepper':
            word2 = 'pepper\'s'
        if word1 not in dict or word2 not in dict:
            continue
        if ('&') in word1 or ('&') in word2:
            continue
        if word1 in dict:
            for j,k in dict[word1].items():
                if j not in bigramCount:
                    for m,n in dict[word2].items():
                        bigramCount[j] = {}
                        temp[j] = {}
                        bigramCount[j][m] = 1
                        temp[j][m] =1
                else:
                    for m,n in dict[word2].items():
                        if m in bigramCount[j]:
                            temp[j][m] +=1
                            bigramCount[j][m] = temp[j][m]/(countTag(dict,j))
                        else:
                            bigramCount[j][m]=1
                            temp[j][m]=1

    return bigramCount


def viterbi(emi, tra, datatest):
    temp = 1
    result = 0
    vit ={}
    taglist = []
    test = nltk.word_tokenize(datatest)
    for i in range(len(test)):
        if test[i] in emi:
            for a,b in emi[test[i]].items():
                if b >1.0:
                    b =1.0
                tag = a
                taglist.append(a)
                vit[test[i]]={}
                vit[test[i]] = a
        elif test[i] not in emi: #Smoothing Unknown word into Noun tag
            vit[test[i]] ={}
            vit[test[i]]['noun']=1.0
            for c,d in vit[test[i]].items():
                tag = c
                taglist.append(c)

        if tag in tra:
            if tag == 'start':
                f = 1
            else:
                for x,y in tra[tag].items():
                    ptag = taglist[i-1]
                    f = tra[ptag][tag]

        temp = temp*f*b
        #print('tem',temp)
    result = temp
    result = result*100 # the result in Percent
    hasil = result
    return hasil,'%'

In [ ]:
listTag = dataread()


In [24]:
d1 = 's.t.r desa saya terletak di kecamatan buahbatu.' #DataTEST
d2 = 's.t.r dia kehabisa pasukan untuk makan.'
transTab = transition(listTag)
emission_probability = emiProb(listTag)
highestProb = postagg(listTag,emission_probability)

vite = viterbi(highestProb,transTab,d1)
vite2 =viterbi(highestProb,transTab,d2)

In [9]:
print('Probability of a word labeled with a tag (emission probability)\n',emission_probability,'\n')

Probability of a word labeled with a tag (emission probability)
 {'s.t.r': {'start': 0.10390946502057613}, 'sembungan': {'propn': 0.5}, 'adalah': {'aux': 0.06395348837209303}, 'sebuah': {'det': 0.07575757575757576}, 'desa': {'noun': 0.0625}, 'yang': {'pron': 0.04195804195804196}, 'terletak': {'verb': 0.05263157894736842}, 'di': {'adp': 0.06222222222222222}, 'kecamatan': {'noun': 0.1111111111111111, 'propn': 0.027777777777777776}, 'kejajar': {'propn': 0.5}, ',': {'punct': 0.05204081632653061}, 'kabupaten': {'noun': 0.04, 'propn': 0.08}, 'wonosobo': {'propn': 0.5}, 'jawa': {'propn': 0.2}, 'tengah': {'propn': 0.07692307692307693}, 'indonesia': {'propn': 0.08433734939759036}, '.': {'punct': 0.06338028169014084}, 'serangan': {'noun': 0.25}, 'pengayauan': {'noun': 0.5}, 'biasanya': {'adv': 0.14285714285714285}, 'terjadi': {'verb': 0.08333333333333333}, 'ladang': {'noun': 0.5}, 'atau': {'cconj': 0.10909090909090909}, 'dengan': {'adp': 0.03888888888888889}, 'membakar': {'verb': 0.25}, 'rumah':

In [10]:
print("The words with each its highest probability",highestProb)

The words with each its highest probability {'s.t.r': {'start': 0.10390946502057613}, 'sembungan': {'propn': 0.5}, 'adalah': {'aux': 0.06395348837209303}, 'sebuah': {'det': 0.07575757575757576}, 'desa': {'noun': 0.0625}, 'yang': {'pron': 0.04195804195804196}, 'terletak': {'verb': 0.05263157894736842}, 'di': {'adp': 0.06222222222222222}, 'kecamatan': {'propn': 0.027777777777777776}, 'kejajar': {'propn': 0.5}, ',': {'punct': 0.05204081632653061}, 'kabupaten': {'propn': 0.08}, 'wonosobo': {'propn': 0.5}, 'jawa': {'propn': 0.2}, 'tengah': {'propn': 0.07692307692307693}, 'indonesia': {'propn': 0.08433734939759036}, '.': {'punct': 0.06338028169014084}, 'serangan': {'noun': 0.25}, 'pengayauan': {'noun': 0.5}, 'biasanya': {'adv': 0.14285714285714285}, 'terjadi': {'verb': 0.08333333333333333}, 'ladang': {'noun': 0.5}, 'atau': {'cconj': 0.10909090909090909}, 'dengan': {'adp': 0.03888888888888889}, 'membakar': {'verb': 0.25}, 'rumah': {'noun': 0.1111111111111111}, 'dan': {'cconj': 0.0488798370672

In [11]:
print('TAG TRANSITION LIST',transTab)

TAG TRANSITION LIST {'start': {'propn': 0.07304526748971193, 'det': 0.030864197530864196, 'noun': 0.11522633744855967, 'adp': 0.12757201646090535, 'sconj': 0.01748971193415638, 'pron': 0.040123456790123455, 'adv': 0.03189300411522634, 'part': 0.00205761316872428, 'verb': 0.0102880658436214, 'cconj': 0.00823045267489712, 'punct': 0.01440329218106996, 'adj': 0.013374485596707819, 'num': 0.00411522633744856}, 'propn': {'aux': 0.012646793134598013, 'propn': 0.20776874435411022, 'punct': 0.2999096657633243, 'pron': 0.044263775971093045, 'cconj': 0.028003613369467027, 'adp': 0.06955736224028906, 'verb': 0.02710027100271003, 'noun': 0.04155374887082204, 'adv': 0.01084010840108401, 'det': 0.018970189701897018, 'adj': 0.01987353206865402, 'num': 0.014453477868112014, 'sym': 0.0018066847335140017, 'sconj': 0.0018066847335140017, 'start': 0.0018066847335140017, 'part': 1}, 'aux': {'det': 0.18023255813953487, 'num': 0.011627906976744186, 'noun': 0.18023255813953487, 'propn': 0.05232558139534884, '

In [23]:
print('Result of data Test->',d1,'=',vite)
print('Result of data Test->',d2,'=',vite2)

Result of data Test-> s.t.r desa saya terletak di kecamatan buahbatu. = (1.6838519051309026e-15, '%')
Result of data Test-> s.t.r dia kehabisa pasukan untuk makan. = (3.97472767801989e-12, '%')
